#<font color = grey> Preparando Ambiente
----


etapa de importação e configuração das bibliotecas necessarias para analizar e tratar os dados.

##Configurando Pyspark

Instalando Dependências

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

Definição das variáveis de ambiente:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

Iniciando o ```findspark```:



In [3]:
import findspark
findspark.init()

Após estas primeiras configurações, podemos iniciar uma SparkSession com os comandos abaixo:

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

download e extração dos arquivos do ngrok:

In [5]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Configurar o nosso Authtoken obtido no site do ngrok:

In [6]:
get_ipython().system_raw('./ngrok authtoken 2IK1NDDOEybXFGDUrhrblkA4ZkI_3KyowVSex27o939tN7WWG')
get_ipython().system_raw('./ngrok http 4050 &')

In [7]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [8]:
spark

##Conectando ao Drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importando Bibliotecas Úteis

In [10]:
from pyspark.sql.functions import sum, col

In [11]:
#Biblioteca para ler .zip
import zipfile

In [12]:
from pyspark.sql.types import DoubleType, StringType, IntegerType
from pyspark.sql import functions as f

#Conectando o Dataframe

##Extranindo arquivo zip

In [13]:
#Extraindo dados
#ZipFile - Local do arquivo para extração, 'r' para apenas ler o arquivo
#extractall - Local para onde quero extrair os arquivos
zipfile.ZipFile('/content/drive/MyDrive/Projeto InsightPlaces/semana-1.zip','r').extractall('/content/drive/MyDrive/Projeto InsightPlaces/Dados Brutos')

##Carregando Dados Brutos
----


[```spark.read.janon```](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrameReader.json.html)

In [14]:
df = spark.read.json('/content/drive/MyDrive/Projeto InsightPlaces/Dados Brutos/dataset_bruto.json')

#<font color = orange>Analizando os Dados

##Tipo dos dados
----

[Base de dados - InsightPlaces](https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip)

### Dicionário de Dados - Anuncio

| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

##Visualizando Dados

In [15]:
df.toPandas()

,anuncio,imagens,usuario
0,"(0, [], [16], [0], [], (Centro, 20061003, Rio ...","[(39d6282a-71f3-47bc-94aa-909351ecd881, https:...","(9d44563d-3405-4e84-9381-35b7cf40a9a4, Frank)"
1,"(0, [], [14], [0], [], (Centro, 20051040, Rio ...","[(23d2b3ab-45b0-47f2-a27f-ffbe066c2c38, https:...","(36245be7-70fe-40cd-84d4-70a57dc8f976, Caroline)"
2,"(0, [1026], [1026], [0], [], (Maria da Graça, ...","[(1da65baa-368b-490f-bef7-1834b9a428c1, https:...","(9dc415d8-1397-4d8d-8425-b8310fad309d, Oliver)"
3,"(0, [120], [120], [0], [Portão eletrônico, Con...","[(79b542c6-49b4-4f06-80b5-ef4fcdf98f76, https:...","(9911a2df-f299-4a73-a384-281417abaf69, Matthew)"
4,"(0, [3], [3], [0], [], (São Cristóvão, 2093167...","[(e2bc497b-6510-4731-8942-97b3d3cdab5e, https:...","(240a7aab-12e5-4069-9a2c-9dca4c4f9c68, Jude)"
...,...,...,...
89078,"(0, [], [219, 283], [4], [], (Barra da Tijuca,...","[(6445e6a2-ab4e-4635-8810-870664488db3, https:...","(ff6637f5-9fb1-4396-adae-327effb5f446, Bruce)"
89079,"(0, [], [200, 446], [3, 4], [], (Leblon, 22450...","[(7d9ccf17-96ab-49db-b601-0f0d5c2f7509, https:...","(9dd42b59-2f28-4022-88a6-4383e7890f73, Tyler)"
89080,"(0, [], [130, 150], [3], [], (Lagoa, 22471004,...","[(03b6e9aa-d622-458b-8fea-b7a708a905ee, https:...","(a06a18b3-4993-4f4b-b2e2-3e9e43fb9d3f, Malcolm)"
89081,"(0, [268], [268], [5], [], (Engenho de Dentro,...","[(0317b2ab-ddc8-45a7-8d96-583c793eb7d7, https:...","(f9292bf8-8f3f-476f-99fe-77caa979e99c, Adele)"


In [16]:
df.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

## Selecionar Coluna Anuncio
----



Para nossa análise, apenas as informações do campo "anuncio" serão relevantes. Por isso, vamos focar em analisar as colunas desse campo.

In [17]:
# Seleciona apenas a coluna "anuncio"
anuncio= df.select('anuncio.*')
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [18]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

##Filtrar a base de dados

----
O time de Data Science solicitou que fizéssemos alguns filtros nas colunas `tipo_uso`, `tipo_unidade` e `tipo_anuncio` da nossa base de dados:

1. tipo_uso: **Residencial**;
2. tipo_unidade: **Apartamento**;
3. tipo_anuncio: **Usado**.





### Entendendo Impactos dos dados
----
Analizando a frequência de cada um dos campos a serem filtrados

####**`tipo_uso`:**

In [19]:
freq_tipo_uso = anuncio.groupBy('tipo_uso').count()
freq_tipo_uso.show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [20]:
# Calcula a contagem de cada tipo de uso
tipo_uso_count = anuncio.groupBy('tipo_uso').count()

# Calcula o total de todas as linhas da coluna de contagem
total = tipo_uso_count.agg(sum('count')).collect()[0][0]

# Adiciona uma nova coluna com a porcentagem de cada tipo de uso em relação ao total
tipo_uso_pct = tipo_uso_count.withColumn('percent', col('count')/total*100)

tipo_uso_pct.show()

+-----------+-----+-----------------+
|   tipo_uso|count|          percent|
+-----------+-----+-----------------+
|  Comercial| 4542|5.098615897533761|
|Residencial|84541|94.90138410246624|
+-----------+-----+-----------------+



Com base nessas informações, podemos analisar a distribuição dos imóveis entre os tipos de uso residencial e comercial. Podemos verificar que a maioria dos imóveis são residenciais, representando cerca de 95% da base de dados, enquanto os imóveis comerciais representam apenas cerca de 5%. Essa análise pode ser útil para compreender a dinâmica do mercado imobiliário da região em que a base de dados foi coletada, bem como para direcionar estratégias de marketing e vendas de imóveis residenciais e comerciais.

####**`tipo_unidade`:**

In [21]:
# Calcula a contagem de cada tipo de unidade
tipo_unidade_count = anuncio.groupBy('tipo_unidade').count()

# Calcula o total de todas as linhas da coluna de contagem
total = tipo_unidade_count.agg(sum('count')).collect()[0][0]

# Adiciona uma nova coluna com a porcentagem de cada tipo de unidade em relação ao total
tipo_unidade_pct = tipo_unidade_count.withColumn('percent', col('count')/total*100)

tipo_unidade_pct.show()


+------------+-----+------------------+
|tipo_unidade|count|           percent|
+------------+-----+------------------+
|      Outros|11963|13.429049313561512|
| Apartamento|66801| 74.98737132786279|
|        Casa|10319| 11.58357935857571|
+------------+-----+------------------+



Uma hipótese é que a predominância de apartamentos no Rio de Janeiro seja explicada pelo fato de que a cidade é uma metrópole, com uma alta densidade populacional e uma grande demanda por moradia. O fato de que o crescimento vertical é mais viável em áreas urbanas também pode ter um papel importante nessa tendência.

####**`tipo_anuncio`:**

In [22]:
# Calcula a contagem de cada tipo de anuncio
tipo_anuncio_count = anuncio.groupBy('tipo_anuncio').count()

# Calcula o total de todas as linhas da coluna de contagem
total = tipo_anuncio_count.agg(sum('count')).collect()[0][0]

# Adiciona uma nova coluna com a porcentagem de cada tipo de anuncio em relação ao total
tipo_anuncio_pct = tipo_anuncio_count.withColumn('percent', col('count')/total*100)

tipo_anuncio_pct.show()


+------------+-----+-------------------+
|tipo_anuncio|count|            percent|
+------------+-----+-------------------+
|       Usado|88827|  99.71262754958859|
|  Lançamento|  256|0.28737245041141407|
+------------+-----+-------------------+



Os dados sugerem que a maioria dos imóveis anunciados são usados, indicando uma maior oferta de imóveis usados no mercado imobiliário. O número de lançamentos é relativamente baixo em comparação com o número de imóveis usados, o que pode indicar uma menor oferta de imóveis novos na região analisada e uma menor oferta de opções para compradores que buscam imóveis novos em relação aos que buscam imóveis usados.

###Aplicar filtros a base
----

In [23]:
anuncio = anuncio\
    .where(anuncio['tipo_uso'].like('Residencial'))\
    .where(anuncio['tipo_unidade'].like('Apartamento'))\
    .where(anuncio['tipo_anuncio'].like('Usado'))

###Verificar filtros a base
----

In [24]:
anuncio\
    .select('tipo_uso')\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|66562|
+-----------+-----+



In [25]:
anuncio\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



In [26]:
anuncio\
    .select('tipo_anuncio')\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



In [27]:
anuncio.show(15)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

##Tratando os Dados

### Transformar Tipos
----
Transformar as colunas dos cômodos  e area dos imóveis de **listas** para **inteiros**

#### verificação das colunas
----
verificação se em cada uma das colunas existe apenas um valor dentro de cada lista

In [28]:
to_int = ['area_total', 'area_util', 'banheiros', 'quartos', 'suites', 'vaga' ]

for coluna in to_int:
    anuncio\
        .select(f.size(col(coluna)).alias(coluna))\
        .groupBy(coluna)\
        .count().alias("Quantidade")\
        .show()

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+



#### Transformando colunas

#####Extranindo elementos das listas

In [29]:
anuncio = anuncio\
    .select([f.col(c)[0].alias(c) if c in to_int else c for c in anuncio.columns])

list comprehension:

`[f.col(c)[0].alias(c) if c in to_int else c for c in anuncio.columns]`

* <font color = purple>`for` <font color = black> **`c`** <font color = blue>`in` <font color = black> **`anuncio.columns`**

    percorrendo todas as colunas do data frame anuncios

* <font color = purple>`if` <font color = black>**`c`** `in` <font color = black>**`to_int`**

    verifica se o valor esta dentro da lista de transformação

    * Se <font color = green>Sim - <font color = black>`f.col(c)[0].alias(c)`:

        seleciona apenas o primeiro elemento de dentro da lista e atribui ele a culuna, tira da lista.
    * Se <font color = red> Não - <font color = black> `else c`
        
        mantem coluna



#####Transformando tipo para inteiro

In [30]:
for coluna in to_int:
    anuncio\
        .select(coluna)\
        .printSchema()

root
 |-- area_total: string (nullable = true)

root
 |-- area_util: string (nullable = true)

root
 |-- banheiros: long (nullable = true)

root
 |-- quartos: long (nullable = true)

root
 |-- suites: long (nullable = true)

root
 |-- vaga: long (nullable = true)



In [31]:
for coluna in to_int:
    anuncio = anuncio\
                .withColumn(coluna,anuncio[coluna].cast(IntegerType()))

In [32]:
for coluna in to_int:
    anuncio\
        .select(coluna)\
        .printSchema()

root
 |-- area_total: integer (nullable = true)

root
 |-- area_util: integer (nullable = true)

root
 |-- banheiros: integer (nullable = true)

root
 |-- quartos: integer (nullable = true)

root
 |-- suites: integer (nullable = true)

root
 |-- vaga: integer (nullable = true)



In [33]:
anuncio = anuncio\
            .withColumn('andar',anuncio['andar'].cast(IntegerType()))

### Tratamento de informações sobre localização

A equipe de ciência de dados nos solicitou que **apenas** as informações sobre **bairro** e zona da cidade fossem extraídas.

Então, vamos analisar a coluna endereco e transformar apenas as informações sobre bairro e zona em colunas de nosso DataFrame.

In [34]:
anuncio\
    .select('endereco')\
    .printSchema()

root
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)



In [35]:
anuncio = anuncio\
            .withColumn('bairro', anuncio['endereco']['bairro'])\
            .withColumn('zona', anuncio['endereco']['zona'])

In [36]:
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|Paciência|Zo

###Transformar cada campo da coluna "valores" em uma coluna separada

#### verificando tipos

In [37]:
anuncio\
    .select('valores')\
    .printSchema()

root
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)



In [38]:
to_column = ['condominio', 'iptu', 'tipo', 'valor']

####extraindo colunas

In [39]:
for c in to_column:
        anuncio = anuncio\
                    .withColumn(c, anuncio['valores'][c][0])

In [40]:
anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|

In [41]:
for coluna in to_column:
    anuncio\
        .select(coluna)\
        .printSchema()

root
 |-- condominio: string (nullable = true)

root
 |-- iptu: string (nullable = true)

root
 |-- tipo: string (nullable = true)

root
 |-- valor: string (nullable = true)



#### Alterando o Tipo

In [42]:
    anuncio = anuncio\
                .withColumn('condominio',anuncio['condominio'].cast(DoubleType()))\
                .withColumn('iptu',anuncio['iptu'].cast(DoubleType()))\
                .withColumn('valor',anuncio['valor'].cast(DoubleType()))

In [43]:
for coluna in to_column:
    anuncio\
        .select(coluna)\
        .printSchema()

root
 |-- condominio: double (nullable = true)

root
 |-- iptu: double (nullable = true)

root
 |-- tipo: string (nullable = true)

root
 |-- valor: double (nullable = true)



### Filtro Vendas


A **InsightPlaces** permite que o(a) anunciante crie um anúncio com duas opções de valor. Assim, o(a) cliente pode criar um anúncio que mostre tanto o valor de venda do imóvel quanto o seu valor de locação, juntamente com os valores de taxa de condomínio (quando houver) e taxa de IPTU. Estes valores são diferenciados pelo campo `tipo` que pode assumir os valores `Venda` e `Aluguel`.

Como se trata de um estudo sobre o preço de venda dos imóveis, o time de cientistas de dados solicitou apenas as informações do tipo VENDA.

Selecione apenas os valores de **Venda**.

In [44]:
anuncio = anuncio\
            .where(anuncio['tipo'].like('Venda'))

In [45]:
anuncio\
    .select('tipo')\
    .groupBy('tipo')\
    .count()\
    .show()

+-----+-----+
| tipo|count|
+-----+-----+
|Venda|66348|
+-----+-----+



In [46]:
anuncio.printSchema()

root
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true

##Excluindo colunas desnecessarias

In [47]:
drop_columns = ['endereco', 'valores']

In [48]:
for c in drop_columns:
    anuncio = anuncio\
                .drop(c)


In [49]:
anuncio.printSchema()

root
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



In [50]:
anuncio.toPandas()

,andar,area_total,area_util,banheiros,caracteristicas,id,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,bairro,zona,condominio,iptu,tipo,valor
0,3,43.0,43,1,"[Academia, Churrasqueira, Playground, Salão de...",d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3,2,NaN,Usado,Apartamento,Residencial,1.0,Paciência,Zona Oeste,245.0,NaN,Venda,15000.0
1,2,42.0,42,1,"[Churrasqueira, Playground, Salão de festas, C...",085bab2c-87ad-452a-bd0f-8b1451509f84,2,NaN,Usado,Apartamento,Residencial,1.0,Paciência,Zona Oeste,0.0,0.0,Venda,15000.0
2,1,41.0,41,1,"[Portaria 24h, Condomínio fechado, Playground,...",18d22cbe-1b86-4764-8def-70c615f15a61,2,NaN,Usado,Apartamento,Residencial,1.0,Guaratiba,Zona Oeste,0.0,0.0,Venda,20000.0
3,3,43.0,43,1,"[Churrasqueira, Piscina, Playground, Salão de ...",bed8a354-9317-4426-b27e-1c798f864271,2,NaN,Usado,Apartamento,Residencial,0.0,Cosmos,Zona Oeste,285.0,NaN,Venda,20000.0
4,2,43.0,43,1,"[Academia, Churrasqueira, Playground, Salão de...",12a13315-d67f-48f0-9497-017b83252781,2,NaN,Usado,Apartamento,Residencial,1.0,Guaratiba,Zona Oeste,245.0,NaN,Venda,15000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66343,20,648.0,648,6,"[Academia, Churrasqueira, Piscina, Salão de fe...",5831f3d1-e004-414e-b717-2f346a440295,5,5.0,Usado,Apartamento,Residencial,4.0,Barra da Tijuca,Zona Oeste,9654.0,44629.0,Venda,13785527.0
66344,20,NaN,648,6,"[Academia, Churrasqueira, Piscina, Playground,...",8577a5cf-b6c7-4b67-9f51-9dce456a493a,5,5.0,Usado,Apartamento,Residencial,5.0,Barra da Tijuca,Zona Oeste,8385.0,44629.0,Venda,13785527.0
66345,20,648.0,648,7,"[Academia, Condomínio fechado, Elevador, Pisci...",db22bde4-838a-4ff1-8e85-cac62fd159f9,5,5.0,Usado,Apartamento,Residencial,5.0,Barra da Tijuca,Zona Oeste,10200.0,39570.0,Venda,13785527.0
66346,1,264.0,264,2,"[Condomínio fechado, Portão eletrônico, Portar...",382599ac-7751-4ade-8a57-bb9633579a46,3,1.0,Usado,Apartamento,Residencial,3.0,Ipanema,Zona Sul,3970.0,3200.0,Venda,13900000.0


#Salvando os Dados

##Arquivos PARQUET

In [51]:
anuncio.write.parquet(
    path='/content/drive/MyDrive/Projeto InsightPlaces/Dados Tratados/parquet',
    mode= 'overwrite'
)

##Arquivo csv

In [ ]:
anuncio.write.csv(
    path='/content/drive/MyDrive/Projeto InsightPlaces/Dados Tratados/csv',
    mode= 'overwrite',
    sep=';',
    header=True
)

##Comparando a velocidade de agregação em cada um desses arquivos.

In [ ]:
%%time
df_parquet = spark.read.parquet('/content/drive/MyDrive/Projeto InsightPlaces/Dados Tratados/parquet')

CPU times: user 4.36 ms, sys: 0 ns, total: 4.36 ms
Wall time: 274 ms


In [ ]:
%%time
df_csv = spark.read.csv(
    '/content/drive/MyDrive/Projeto InsightPlaces/Dados Tratados/csv', 
    sep=';', 
    inferSchema=True,
    header=True
)

CPU times: user 16.6 ms, sys: 1.7 ms, total: 18.3 ms
Wall time: 1.66 s


Podemos concluir que a leitura do arquivo em formato Parquet é consideravelmente mais rápida do que a leitura do arquivo em formato CSV. Isso pode ser explicado pelo fato de que o Parquet é um formato de armazenamento colunar, o que o torna mais eficiente na leitura e agregação de dados, enquanto o CSV é um formato de armazenamento em linhas, o que pode tornar a leitura e agregação de dados mais lenta. Portanto, se a velocidade de leitura e agregação de dados for um fator importante, o uso de arquivos Parquet pode ser mais vantajoso.

#Encerrando Sessão

In [52]:
spark.stop()